# SMS Classification



In [1]:
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer
from nlp import load_dataset, Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
sms = pd.read_csv("../corpus/sg_sms_corpus_en.csv")

In [3]:
sms.shape

(44097, 2)

In [4]:
sms.head()

,user_id,text
0,0,Bugis oso near wat...
1,0,"Go until jurong point, crazy.. Available only ..."
2,0,I dunno until when... Lets go learn pilates...
3,0,Den only weekdays got special price... Haiz......
4,0,Meet after lunch la...


In [5]:
# unique user_id to predict
sms.user_id.nunique()

59

In [6]:
# stratified sampling for train-test split
sms_train = pd.DataFrame()
for user_id in sms.user_id.unique():
    sms_train = pd.concat([sms_train, sms[sms.user_id == user_id].sample(frac=0.7)])

In [7]:
sms_train.shape

(30867, 2)

In [8]:
sms_test = sms.drop(sms_train.index)

In [9]:
sms_train.reset_index(drop=True, inplace=True)
sms_test.reset_index(drop=True, inplace=True)

In [10]:
sms_test.shape

(13230, 2)

In [11]:
# make sure no overlap of sms in train-test
sms_train.text.isin(sms_test.text).sum()

0

In [12]:
train_dataset = Dataset.from_pandas(sms_train)
test_dataset = Dataset.from_pandas(sms_test)

In [13]:
train_dataset = train_dataset.map(lambda examples: {"label": examples["user_id"]}, batched=True)
test_dataset = test_dataset.map(lambda examples: {"label": examples["user_id"]}, batched=True)

In [14]:
# allow up to 10 mins to download the model when running for the first time
tokenizer = BertTokenizer.from_pretrained("zanelim/singbert-large-sg")
model = BertForSequenceClassification.from_pretrained("zanelim/singbert-large-sg", 
                                                      num_labels=sms_train.user_id.nunique())

Some weights of the model checkpoint at zanelim/singbert-large-sg were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [15]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

In [16]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

In [17]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [18]:
# freeze weights of pre-trained model
for param in model.base_model.parameters():
    param.requires_grad = False

In [19]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [20]:
training_args = TrainingArguments(
    output_dir="../data/sms_classification",
    num_train_epochs=6,
    per_device_train_batch_size=200,
    per_device_eval_batch_size=64,
    warmup_steps=300,
    weight_decay=0.01,
    logging_dir="../data/sms_classification",
    save_steps=5000,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [21]:
# this will take around 1 hour on 2 GPUs
trainer.train()

/home/zane/miniconda3/envs/nlp/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:26: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


/home/zane/miniconda3/envs/nlp/lib/python3.8/site-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)
/home/zane/miniconda3/envs/nlp/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=468, training_loss=3.593337511914408)

In [22]:
trainer.evaluate()

/home/zane/miniconda3/envs/nlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 3.359514885223829,
 'eval_accuracy': 0.17936507936507937,
 'eval_f1': 0.08312945202292296,
 'eval_precision': 0.07306547096503462,
 'eval_recall': 0.17936507936507937,
 'epoch': 6.0}